In [2]:
# Inspired from https://github.com/openfisca/openfisca-france/issues/1385#issuecomment-569054638

In [3]:
from pathlib import Path
import re
import json

In [4]:
# 1. Do: git clone https://github.com/openfisca/openfisca-france.git
# 2. Put the path to the repo directory here:
OPENFISCA_PATH = "../openfisca-france/"

In [7]:
cerfa_fields = {}
for path in Path(OPENFISCA_PATH).glob('**/*.py'):
    with path.open('r') as f:
        content = f.read()
        all_classes = re.findall(r"class (.+?)\(", content)
        for cls in all_classes:
            cls_content = re.findall(r"class " + cls + "\((.*?)class", content.replace("\n", "\t") + "class")[0]
            # Remove comments
            cls_content = re.sub(r"#([^\n\t]+)", r" ", cls_content)
            cerfa_field = re.findall(r'cerfa_field\s+=\s+["\'](\w+)["\']', cls_content)
            assert len(cerfa_field) <= 1
            if cerfa_field:
                cerfa_fields[cls] = [cerfa_field[0]]
            cerfa_field = re.findall(r'cerfa_field\s*=\s*{([^}]+)', cls_content)
            assert len(cerfa_field) <= 1
            if cerfa_field:
                boxes = re.findall(r'["\'][0-9A-Z]{3,4}["\']', cerfa_field[0])
                assert boxes
                cerfa_fields[cls] = [box.strip('\'" ') for box in boxes]

In [145]:
boxes_common = []
mapping_partner = {}
mapping_children = {}

for box_group, boxes in cerfa_fields.items():
    reference_box = boxes[0]
    if len(boxes) == 1:
        boxes_common.append(reference_box)
    elif len(boxes) >= 2:
        mapping_partner[boxes[1]] = reference_box
        for child_box in sorted(boxes[2:]):
            mapping_children[child_box] = reference_box    

In [150]:
with open('boxes.json', 'w+') as f:
    f.write(json.dumps({
        "common": boxes_common,
        "partner_mapping": mapping_partner,
        "children_mapping": mapping_children,
    }))